The aim is to read plot EFE

1) Adam, O., T. Bischoff, and T. Schneider, 2016: Seasonal and Interannual Variations of the Energy Flux Equator and ITCZ. Part I: Zonally Averaged ITCZ Position. J. Climate, 29, 3219–3230, https://doi.org/10.1175/JCLI-D-15-0512.1.
2) EFPM : NOT USING NOW. keeping the reference here: Boos, W., Korty, R. Regional energy budget control of the intertropical convergence zone and application to mid-Holocene rainfall. Nature Geosci 9, 892–897 (2016). https://doi.org/10.1038/ngeo2833


In [ ]:
import numpy as np
import xarray as xr
import matplotlib as mpl
from pathlib import Path

# our local module:
import itcz
import myfunctions as mf
import mse_divergence as mdiv 

In [ ]:
# =========================
# User-defined metadata
# =========================

# Base CEDA paths
CEDA_BASE = Path("/badc/cmip6/data/CMIP6")

#Model Names
MODELS = {
    "UKESM1-0-LL":  {"institution": "MOHC",         "ensemble": "r1i1p1f2",  "grid": "gn",},
    # "CNRM-ESM2-1":  {"institution": "CNRM-CERFACS", "ensemble": "r1i1p1f2",  "grid": "gr",},
    # "MPI-ESM1-2-LR":{"institution": "MPI-M",        "ensemble": "r1i1p1f1",  "grid": "gn",},
    # "CESM2-WACCM":  {"institution": "NCAR",         "ensemble": "r1i1p1f1",  "grid": "gn",},
    # "IPSL-CM6A-LR": {"institution": "IPSL",         "ensemble": "r1i1p1f1",  "grid": "gr",},
}

#Experiment details
EXPERIMENTS = {
    # "HIST":     {"project": "CMIP",        "scenario": "historical", "color": "black"},
    "SSP245":   {"project": "ScenarioMIP", "scenario": "ssp245"},
    "SSP585":   {"project": "ScenarioMIP", "scenario": "ssp585"},
    "G6solar":  {"project": "GeoMIP",      "scenario": "G6solar"},
    "G6sulfur": {"project": "GeoMIP",      "scenario": "G6sulfur"},
}




In [ ]:
import itcz
ITCZ_Adam_UKESM_2021_2050 = itcz.compute_adam_ITCZ(
    model_name="UKESM1-0-LL",
    model_meta=MODELS["UKESM1-0-LL"],
    experiments=EXPERIMENTS,
    base_dir=CEDA_BASE,
    lon_slice=slice(0, 360),
    season=(1, 12),
    time_slice=slice(2021, 2051),
)

In [ ]:
import itcz
ITCZ_Adam_UKESM_2071_2100 = itcz.compute_adam_ITCZ(
    model_name="UKESM1-0-LL",
    model_meta=MODELS["UKESM1-0-LL"],
    experiments=EXPERIMENTS,
    base_dir=CEDA_BASE,
    lon_slice=slice(0, 360),
    season=(1, 12),
    time_slice=slice(2071, 2101),
)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

experiments = ["SSP245", "G6sulfur", "G6solar"]

fig, axes = plt.subplots(
    len(experiments), 1,
    figsize=(14, 10),
    sharex=True,
    subplot_kw=dict(projection=ccrs.PlateCarree())
)

for ax, exp in zip(axes, experiments):

    da = ITCZ_Adam_UKESM_2071_2100[exp]   # (year, lon)

    # --- mean & std over years ---
    mean = da.mean("year")
    std  = da.std("year")

    # --- wrap longitudes to [-180, 180] ---
    lon = mean.lon.values
    lon_wrap = ((lon + 180) % 360) - 180
    sort_idx = np.argsort(lon_wrap)

    lon_sorted  = lon_wrap[sort_idx]
    mean_sorted = mean.values[sort_idx]
    std_sorted  = std.values[sort_idx]

    # --- plot mean ITCZ ---
    ax.plot(
        lon_sorted,
        mean_sorted,
        color="C0",
        linewidth=2,
        transform=ccrs.PlateCarree(),
        label="Mean ITCZ"
    )

    # --- ±1σ shading ---
    ax.fill_between(
        lon_sorted,
        mean_sorted - std_sorted,
        mean_sorted + std_sorted,
        color="C0",
        alpha=0.3,
        transform=ccrs.PlateCarree(),
        label="±1σ"
    )

    # --- reference lines ---
    ax.axhline(0, color="k", linewidth=0.6, linestyle="--")

    # --- map features ---
    ax.set_extent([-180, 180, -30, 30], crs=ccrs.PlateCarree())
    ax.coastlines(linewidth=0.6)
    ax.add_feature(cfeature.LAND, facecolor="lightgray", zorder=0)
    ax.add_feature(cfeature.OCEAN, facecolor="aliceblue", zorder=0)

    ax.set_title(f"{exp} (2071–2100)")

axes[-1].set_xlabel("Longitude (°)")
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

experiments = ["SSP245", "G6sulfur", "G6solar"]
colors = {
    "SSP245": "C0",        # blue
    "G6sulfur": "C2",      # green
    "G6solar": "pink"
}

datasets = {
    "2021–2050": ITCZ_Adam_UKESM_2021_2050,
    "2071–2100": ITCZ_Adam_UKESM_2071_2100
}

fig, axes = plt.subplots(
    2, 1,
    figsize=(14, 8),
    sharex=True,
    subplot_kw=dict(projection=ccrs.PlateCarree())
)

fig.subplots_adjust(
    hspace=-0.10,   # vertical spacing between rows
    wspace=0.02,   # horizontal spacing between columns
    bottom=0.12    # room for colorbar
)

for ax, (period, data) in zip(axes, datasets.items()):

    for exp in experiments:

        da = data[exp]  # (year, lon)

        # mean & std over years
        mean = da.mean("year")
        std  = da.std("year")

        # wrap longitudes to [-180, 180]
        lon = mean.lon.values
        lon_wrap = ((lon + 180) % 360) - 180
        sort_idx = np.argsort(lon_wrap)

        lon_sorted  = lon_wrap[sort_idx]
        mean_sorted = mean.values[sort_idx]
        std_sorted  = std.values[sort_idx]

        # plot mean
        ax.plot(
            lon_sorted,
            mean_sorted,
            color=colors[exp],
            linewidth=2,
            label=exp,
            transform=ccrs.PlateCarree()
        )

        # ±1σ shading
        ax.fill_between(
            lon_sorted,
            mean_sorted - std_sorted,
            mean_sorted + std_sorted,
            color=colors[exp],
            alpha=0.25,
            transform=ccrs.PlateCarree()
        )

    # reference line
    ax.axhline(0, color="k", linewidth=0.6, linestyle="--")

    # map cosmetics
    ax.set_extent([-180, 180, -30, 30], crs=ccrs.PlateCarree())
    ax.coastlines(linewidth=0.6)
    ax.add_feature(cfeature.LAND, facecolor="lightgray", zorder=0)
    ax.add_feature(cfeature.OCEAN, facecolor="aliceblue", zorder=0)

    ax.set_title(f"Rainfall centroid – {period}")
    ax.legend(loc="upper right", frameon=False)

axes[-1].set_xlabel("Longitude (°)")
plt.tight_layout()

# Save
fig_directory = "/home/users/bidyut/20260112_Basic_Analysis"
figname = "Figure_ITCZ_centroid_overlay_UKESM"
# # plt.savefig(''+str(fig_directory)+'/'+str(figname)+'.svg', format="svg",transparent=True, dpi=1200)
# # !rsvg-convert -f pdf -o {fig_directory}/{figname}.pdf {fig_directory}/{figname}.svg
plt.savefig(f"{fig_directory}/{figname}.pdf", format="pdf")
plt.show()